In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Load BCG TF image dataset
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define Hpt-Net architecture
def hyp_net(input_shape):
    inputs = Input(shape=input_shape)

    # Convolutional Layers
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    # Flatten the output of the convolutional layers
    x = Flatten()(x)

    # Fully Connected Layers
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output Layer
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    return model

# Create Hyp-Net model
model = hyp_net(input_shape=(224, 224, 3))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Define learning rate scheduler and early stopping
lr_scheduler = ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

epochs = 20  # Define the number of training epochs

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_accuracy}")

predictions = model.predict(test_generator)
predicted_classes = np.round(predictions).astype(int).flatten()
true_classes = test_generator.classes

# Classification report and confusion matrix
print(f"\nClassification Report:\n")
print(classification_report(true_classes, predicted_classes))
print(f"\nConfusion Matrix:\n")
print(confusion_matrix(true_classes, predicted_classes))

# Separate the training and validation data
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Plot validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(val_accuracy, marker='x', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy per Epoch')
plt.legend()
plt.show()

# Plot validation loss
plt.figure(figsize=(12, 6))
plt.plot(val_loss, marker='o', color='r', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss per Epoch')
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Load BCG TF image dataset
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


# Increase Model Capacity
def hyp_net_with_resnet50(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)  # Increased capacity
    x = Dropout(0.5)(x)  # Reduced dropout rate
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    return model

# Create Hyp-Net model with ResNet50
model = hyp_net_with_resnet50(input_shape=(224, 224, 3))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 20  # Define the number of training epochs

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[ ReduceLROnPlateau(factor=0.2, patience=3)],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_accuracy}")

predictions = model.predict(test_generator)
predicted_classes = np.round(predictions)
true_classes = test_generator.classes

# Classification report and confusion matrix
print(f"\nClassification Report:\n")
print(classification_report(true_classes, predicted_classes))
print(f"\nConfusion Matrix:\n")
print(confusion_matrix(true_classes, predicted_classes))


# Separate the training and validation data
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Plot validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(val_accuracy, marker='x', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy per Epoch')
plt.legend()
plt.show()

# Plot validation loss
plt.figure(figsize=(12, 6))
plt.plot(val_loss, marker='o', color='r', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss per Epoch')
plt.legend()
plt.show()



In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Load BCG TF image dataset
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/dataset/MM_project/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define AlexNet model
def alexnet(input_shape):
    inputs = Input(shape=input_shape)
    
    # First convolutional layer
    x = Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    # Second convolutional layer
    x = Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    # Third, Fourth, Fifth convolutional layers
    x = Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    # Flatten
    x = Flatten()(x)
    
    # First fully connected layer
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Second fully connected layer
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Output layer
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    return model

# Create AlexNet model
model = alexnet(input_shape=(224, 224, 3))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 20  # Define the number of training epochs

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[ReduceLROnPlateau(factor=0.2, patience=3)],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_accuracy}")

predictions = model.predict(test_generator)
predicted_classes = np.round(predictions)
true_classes = test_generator.classes

# Classification report and confusion matrix
print(f"\nClassification Report:\n")
print(classification_report(true_classes, predicted_classes))
print(f"\nConfusion Matrix:\n")
print(confusion_matrix(true_classes, predicted_classes))

# Separate the training and validation data
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Plot validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(val_accuracy, marker='x', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy per Epoch')
plt.legend()
plt.show()

# Plot validation loss
plt.figure(figsize=(12, 6))
plt.plot(val_loss, marker='o', color='r', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss per Epoch')
plt.legend()
plt.show()


2024-06-03 04:24:32.620574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 04:24:32.620674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 04:24:32.787497: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


KeyboardInterrupt: 